<a href="https://colab.research.google.com/github/javavisa12/ACE---Vizdetect/blob/main/ACE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install streamlit pandas scikit-learn matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 61.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 79.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.6 MB/s eta 0:00:00


In [3]:
!pip install pyngrok


In [4]:
!wget -q -O ngrok.zip https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-stable-linux-amd64.zip
!unzip -o ngrok.zip
!chmod +x ngrok

Archive:  ngrok.zip
  inflating: ngrok                   


In [5]:
with open("app.py", "w") as f:
    f.write("""
import streamlit as st
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# Title
st.title("🔍 One-Click Anomaly Detection System")

# Upload CSV File
uploaded_file = st.file_uploader("Upload your transaction data (CSV)", type="csv")

if uploaded_file:
    # Load Data
    st.subheader("📊 Uploaded Data")
    df = pd.read_csv(uploaded_file)
    st.write(df.head())

    # Feature Selection
    st.subheader("⚙️ Selecting Relevant Features")
    features = ['Time', 'Amount']
    if all(col in df.columns for col in features):
        X = df[features]

        # Preprocess the Data
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)

        # Train Isolation Forest
        st.subheader("🚀 Running Anomaly Detection")
        isolation_forest = IsolationForest(n_estimators=100, contamination=0.01, random_state=42)
        predictions = isolation_forest.fit_predict(X_scaled)
        df['Anomaly'] = predictions
        df['Anomaly'] = df['Anomaly'].map({1: 'Normal', -1: 'Anomaly'})

        # Show Results
        st.subheader("✅ Results")
        st.write(df.head(10))

        # Summary
        anomaly_count = df['Anomaly'].value_counts()
        st.write("Anomalies Detected:")
        st.write(anomaly_count)

        # Visualization
        st.subheader("📈 Anomaly Visualization")
        plt.figure(figsize=(10, 6))
        plt.scatter(df['Time'], df['Amount'], c=(df['Anomaly'] == 'Anomaly'), cmap='coolwarm', alpha=0.6)
        plt.title("Anomaly Detection: Time vs Amount")
        plt.xlabel("Time")
        plt.ylabel("Amount")
        st.pyplot(plt)

    else:
        st.error("The dataset must contain 'Time' and 'Amount' columns.")""")



In [ ]:
from pyngrok import ngrok




In [6]:
!./ngrok config add-authtoken 2qInwrnb5yMXRvfw3kn6Les0urY_552jUEHErgJU8uPYuEXxi


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [7]:
from pyngrok import ngrok

# Start the ngrok tunnel using the correct configuration format
tunnel = ngrok.connect(8501, "http")  # Expose HTTP on port 8501
print("Streamlit is running at:", tunnel)

# Step 6: Run the Streamlit app
!streamlit run app.py &>/dev/null &

Streamlit is running at: NgrokTunnel: "https://69cc-35-203-188-60.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
!streamlit run app.py &>/dev/null &

In [ ]:
import os
import time
from pyngrok import ngrok

def start_tunnel():
    # Start the Streamlit app
    os.system("streamlit run app.py &")

    # Start the ngrok tunnel
    public_url = ngrok.connect(8501)  # Adjust the port if needed
    print(f"ngrok tunnel established at: {public_url}")
    return public_url

def monitor_tunnel():
    try:
        tunnel = start_tunnel()
        while True:
            # Keep the script running to monitor the tunnel
            time.sleep(300)  # Check every 5 minutes
    except KeyboardInterrupt:
        print("Shutting down...")
        ngrok.kill()

if __name__ == "__main__":
    monitor_tunnel()
